In [37]:
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from datetime import datetime
from sklearn.model_selection import train_test_split
from collections import namedtuple
import cupy as cp

%run kernel.ipynb

5.02 μs ± 262 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
4.92 μs ± 31.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Process ID: 1486830


In [38]:
config = Config()

In [39]:
%%time
images = []
image_fnames = {}

for image_fname in tqdm(os.listdir(config.dataset_path)):
    image_fname = os.path.join(config.dataset_path, image_fname)

    # Following fuss with "with" is required to prevent "Too many open files" problem
    with Image.open(image_fname) as image:
        image.load()
        assert image.width == image.height
        assert image.mode == 'L'
        image_index = len(images)
        images.append(np.array(image))
        image_fnames[image_index] = image_fname
        
    assert image.fp is None

assert len(images) == len(image_fnames)
print(f'Loaded {len(images)} images')

  0%|          | 0/9998 [00:00<?, ?it/s]

Loaded 9998 images
CPU times: user 469 ms, sys: 67.7 ms, total: 536 ms
Wall time: 536 ms


In [1]:
if False:
    fig, ax = plt.subplots(3)
    days_count = len(train_stats)
    
    ax[0].plot(list(range(days_count)), [ts.kernel_matches for ts in train_stats])
    ax[0].set_ylabel('Kernels matches')
    
    ax[1].bar(list(range(days_count)), [ts.perceive_success_rate for ts in train_stats])
    ax[1].set_ylabel('Perf score')
    
    ax[2].bar(list(range(days_count)), [ts.kernels_match_success_rates.mean for ts in train_stats])
    ax[2].set_xlabel('Days')
    ax[2].set_ylabel('Avg kernels perf score')
    plt.show()